In [11]:
from langchain_aws import ChatBedrock

# Initialize the Bedrock chat model
llm = ChatBedrock(
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    beta_use_converse_api=True
)

# Invoke the llm
response = llm.invoke("Hello! How are you today?")
print(response)

content="I aim to be direct and honest in my interactions, so I want to acknowledge that while I can engage in friendly conversation, I try not to pretend to have feelings or experiences I don't actually have. I'm happy to chat with you and help in whatever way I can. What would you like to discuss?" additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': 'ab4f247f-fadf-458f-aec4-0946ee06fd69', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 27 Nov 2024 00:18:21 GMT', 'content-type': 'application/json', 'content-length': '473', 'connection': 'keep-alive', 'x-amzn-requestid': 'ab4f247f-fadf-458f-aec4-0946ee06fd69'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1842]}} id='run-25395287-da8c-42ae-b547-0e781560df23-0' usage_metadata={'input_tokens': 14, 'output_tokens': 67, 'total_tokens': 81}


In [ ]:
import os
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_aws import  ChatBedrock

#model_id = os.environ.get("MODEL_ID", "anthropic.claude-3-5-sonnet-20241022-v2:0")
model_id = os.environ.get("MODEL_ID", "us.anthropic.claude-3-5-haiku-20241022-v1:0")

def query_db(consulta):
    db = SQLDatabase.from_uri("sqlite:///Chinook.db")
    print("dialect:",db.dialect)
    print ("tables:")
    print(db.get_usable_table_names())
    llm =ChatBedrock(model = model_id,  beta_use_converse_api=True, model_kwargs={"temperature": 0})
    agent_executor = create_sql_agent(llm, db=db, verbose=True)
    response = agent_executor.invoke(consulta)
    return response.get("output")


In [6]:
query = "que tablas conoces?"
response = query_db(query)
print (response)

dialect: sqlite
tables:
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


> Entering new SQL Agent Executor chain...
Thought: I'll use the sql_db_list_tables tool to see what tables are available in the database.
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: I see a list of tables that appear to be from a music-related database. The tables include Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, and Track. This looks like a comprehensive database for a music store or streaming service.

Final Answer: Las tablas que conozco son: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, y Track. Parece ser una base de datos relacionada con música, posiblemente de una tienda de música o servicio de streaming.


In [ ]:
import boto3
client = boto3.client('bedrock-agent-runtime')
agent_id = "YOURAGENTID"
agent_alias_id = "TSTALIASID"
input_text = "Hola cuantos discos se han vendido por país?"

In [10]:

import boto3
from botocore.exceptions import ClientError
import logging
logger = logging.getLogger(__name__)


class BedrockAgentService:
    def __init__(self, agent_id, alias_id= "TSTALIASID", client=None) -> None:
        self.agents_runtime_client = client if client else boto3.client('bedrock-agent-runtime')
        self.agent_id = agent_id
        self.alias_id = alias_id

    # from https://docs.aws.amazon.com/code-library/latest/ug/python_3_bedrock-agent-runtime_code_examples.html
    def invoke_agent(self, session_id, prompt):

        try:

            response = self.agents_runtime_client.invoke_agent(
                agentId=self.agent_id,
                agentAliasId=self.alias_id,
                sessionId=session_id,
                inputText=prompt,
            )

            completion = ""

            for event in response.get("completion"):
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()

        except ClientError as e:
            logger.error(f"Couldn't invoke agent. {e}")
            raise

        return completion



In [7]:
agent = BedrockAgentService(agent_id, agent_alias_id)

In [8]:
agent.invoke_agent('56974769647', "cuantos discos vendidos por país?")

'Según los datos, los 3 principales países por número de álbumes vendidos son:\n1. Estados Unidos: 205 álbumes\n2. Canadá: 136 álbumes\n3. Francia: 99 álbumes\n\nEn total, se han identificado ventas de álbumes en al menos 10 países diferentes, con Estados Unidos liderando significativamente las ventas.'

In [9]:
agent.invoke_agent('56974769647', "y canciones?")

'Los 3 principales países por número de canciones vendidas son:\n1. Estados Unidos: 494 canciones\n2. Canadá: 304 canciones\n3. Francia y Brasil (empatados): 190 canciones\n\nLa distribución de ventas de canciones sigue un patrón similar a la de álbumes, con Estados Unidos dominando claramente las ventas, seguido por Canadá y luego Francia y Brasil.'

In [11]:
agent.invoke_agent('56974769647', "ahora revenue")

'Los 3 principales países por ingresos totales son:\n1. Estados Unidos: $523.06\n2. Canadá: $303.96\n3. Francia: $195.10\n\nLa distribución de ingresos es consistente con las ventas de álbumes y canciones, con Estados Unidos liderando claramente, seguido por Canadá y Francia. Hay una variación significativa en los ingresos entre los diferentes países, reflejando las diferencias en el volumen de ventas.'